In [87]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [88]:
df = pd.read_csv(r"C:\\Users\\as041\\OneDrive\\Desktop\\NLP\\Implementation\\Dataset\\HindiTrainDataset.csv")
df = df.drop(["Unnamed: 0", "total_classes"], axis="columns")
# df = df.drop(["Unnamed: 0", "AuthorName", "AuthorChannelId", "PublishedDate", "LikeCount"], axis="columns")
df

,Comment,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,"This <a href="" will solve everything for long ...",0,0,0,0,0,0
1,Where can I Buy link please 🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣...,0,0,0,0,0,0
2,But i fall in love with you the you deliver th...,0,0,1,0,0,0
3,Apko kiss nhi chahiye kya,1,0,1,0,0,0
4,I just wanna kiss you,0,0,1,0,0,0
...,...,...,...,...,...,...,...
2085,Aap bhi aese hi kapde pahna chalu kr do wow k...,1,0,0,0,0,0
2086,tujhse achi modern to hmari bhais h ek bhi Kap...,0,0,1,0,0,0
2087,Besharam aurat,0,1,0,0,0,1
2088,Aap bohot pyari lag rahi ho,0,0,0,0,0,0


In [89]:
print(df.isna().sum())
df = df.dropna()
df

Comment          13
toxic             0
severe_toxic      0
obscene           0
threat            0
insult            0
identity_hate     0
dtype: int64


,Comment,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,"This <a href="" will solve everything for long ...",0,0,0,0,0,0
1,Where can I Buy link please 🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣...,0,0,0,0,0,0
2,But i fall in love with you the you deliver th...,0,0,1,0,0,0
3,Apko kiss nhi chahiye kya,1,0,1,0,0,0
4,I just wanna kiss you,0,0,1,0,0,0
...,...,...,...,...,...,...,...
2085,Aap bhi aese hi kapde pahna chalu kr do wow k...,1,0,0,0,0,0
2086,tujhse achi modern to hmari bhais h ek bhi Kap...,0,0,1,0,0,0
2087,Besharam aurat,0,1,0,0,0,1
2088,Aap bohot pyari lag rahi ho,0,0,0,0,0,0


In [90]:
df.iloc[3]['Comment']

'Apko kiss nhi chahiye kya'

In [91]:
df[df.columns[1:]].iloc[3]

toxic            1
severe_toxic     0
obscene          1
threat           0
insult           0
identity_hate    0
Name: 3, dtype: int64

Prerprocessing Data -- Making it fit for the Model

In [92]:
# from tensorflow.keras.layers import TextVectorization 
from keras.layers import TextVectorization

In [93]:
X = df['Comment']
y = df[df.columns[1:]].values

In [94]:
MAX_FEATURES = 10000 #number of words in the vocab

In [95]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=500,
                               output_mode='int')


In [96]:
vectorizer.adapt(X.values) #learn all the values of the vocabulary

In [97]:
vectorized_text = vectorizer(X.values)
vectorized_text

<tf.Tensor: shape=(2077, 500), dtype=int64, numpy=
array([[ 132,   61,  294, ...,    0,    0,    0],
       [1022,  333,   39, ...,    0,    0,    0],
       [ 137,   39, 1397, ...,    0,    0,    0],
       ...,
       [1499,  338,    0, ...,    0,    0,    0],
       [  20,  276, 2299, ...,    0,    0,    0],
       [ 627, 4380,   11, ...,    0,    0,    0]], dtype=int64)>

In [98]:
import tensorflow as tf

In [99]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(2000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [100]:
batch_X, batch_y = dataset.as_numpy_iterator().next()

In [101]:
batch_X.shape

(16, 500)

In [102]:
batch_y.shape

(16, 6)

In [103]:
train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [104]:
train_generator = train.as_numpy_iterator().next()

Creating Sequential Model

In [105]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [106]:
model = Sequential()
#create the embedding layer, +1 in max features is for unknown words
model.add(Embedding(MAX_FEATURES+1, 32))
#Creating a LSTM layers with 32 different unit, "tanh" works good with tensorflow
#Bidirectional help the model look after and well before of the word which contain some meaning
model.add(Bidirectional(LSTM(32, activation='tanh')))
#feature extractor fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
#final layer (6 -- because of total 6 output classes)
model.add(Dense(6, activation='sigmoid'))


In [107]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          320032    
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               16640     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 256)               33024     
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dense_7 (Dense)             (None, 6)                 774       
                                                      

In [108]:
history = model.fit(train, epochs = 10, validation_data=val)

Epoch 1/10
91/91 [==============================] - 47s 487ms/step - loss: 0.4342 - val_loss: 0.3866
Epoch 2/10
91/91 [==============================] - 34s 372ms/step - loss: 0.3942 - val_loss: 0.3471
Epoch 3/10
91/91 [==============================] - 31s 346ms/step - loss: 0.3670 - val_loss: 0.3334
Epoch 4/10
91/91 [==============================] - 31s 337ms/step - loss: 0.3354 - val_loss: 0.3474
Epoch 5/10
91/91 [==============================] - 30s 335ms/step - loss: 0.3233 - val_loss: 0.3096
Epoch 6/10
91/91 [==============================] - 30s 332ms/step - loss: 0.3091 - val_loss: 0.3261
Epoch 7/10
91/91 [==============================] - 31s 336ms/step - loss: 0.2984 - val_loss: 0.2857
Epoch 8/10
91/91 [==============================] - 30s 332ms/step - loss: 0.2846 - val_loss: 0.2605
Epoch 9/10
91/91 [==============================] - 30s 333ms/step - loss: 0.2592 - val_loss: 0.2679
Epoch 10/10
91/91 [==============================] - 31s 338ms/step - loss: 0.2478 - val_lo

In [109]:
# saving the model for future use
model.save('hindi_toxicity_model.h5')

Model Evaluation

In [110]:
from keras.metrics import Precision, Recall, CategoricalAccuracy

In [111]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [112]:
for batch in test.as_numpy_iterator():
    # unpack the batch
    X_true, y_true = batch
    # make a prediction
    yhat = model.predict(X_true)
    # flatten the prediction
    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 76ms/step


In [113]:
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}')

Precision: 0.6859503984451294, Recall: 0.40487805008888245, Accuracy: 0.07692307978868484


Model Predictions

In [114]:
model = tf.keras.models.load_model("C:\Z_Sem6_Project\ModelCreation\hindi_toxicity_model.h5", compile= False)

In [115]:
def predict_comment(comment):
  vectorised_comment = vectorizer([comment])
  results = model.predict(vectorised_comment)

  text = ''
  for idx, col in enumerate(df.columns[1:]):
    text += '{}: {}\n'.format(col, results[0][idx]>0.25)

  return results, text

In [116]:
comment = "achhi video hain"
ans = predict_comment(comment)
print("Prediction Results:", ans[0])
print(ans[1])

1/1 [==============================] - 1s 726ms/step
Prediction Results: [[0.54066634 0.10269322 0.40060055 0.0045579  0.01601654 0.0009685 ]]
toxic: True
severe_toxic: False
obscene: True
threat: False
insult: False
identity_hate: False

